In [ ]:
# import pickle
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import scipy.signal as signal
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from paths import path_samoc, path_results, path_CESM
from paths import path_ocn_ctrl, path_ocn_ctrl_rect
from paths import path_ocn_rcp,  path_ocn_rcp_rect
from paths import CESM_filename, file_ex_ocn_rect
from timeseries import IterateOutputCESM, yrly_avg_nc, lowpass, chebychev

# filtering

In [ ]:
b, a = sp.signal.cheby1(4, 1, 100, 'low', analog=True)
w, h = sp.signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)), label='chebychev')
plt.xscale('log')
plt.title(' frequency response (rp=5)')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axhline(0, c='k', lw=.5) # rp

b, a = sp.signal.butter(4, 100, 'low', analog=True)
w, h = sp.signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)), label='butterworth')
plt.xscale('log')
plt.title('Butterworth/Chebychev filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green', label='cutoff frequency')

plt.legend()
plt.tight_layout()

### notch filter

In [ ]:
from timeseries import 

In [ ]:
test = [np.sin(i*np.pi/12) for i in range(240)]
noise = .5*np.random.rand((len(test)))
trend = np.arange(0,1,1/len(test))
test2 = test+noise
test3 = test+noise+trend

In [ ]:
plt.figure(figsize=(8,5))
plt.axhline(0, c='k', lw=.5)
plt.plot(test, label='sin(t)')
plt.plot(trend, label='trend')
plt.plot(noise, label='noise')
plt.plot(noise+trend, label='noise+trend')
plt.plot(test2, label='sin(t)+noise')
plt.plot(test3, label='sin(t)+noise+trend')
# plt.plot(notch(test,12))
plt.plot(notch(test2,12), label='notch(sin+noise)')
plt.plot(notch(test3,12), label='notch(sin+noise+trend)')
plt.legend()

deseasonalizing example: see SST_PDO.ipynb

# averaging timeseries and creating new netCDF files

In [ ]:
# len(IterateOutputCESM(domain='atm', run='lpi', tavg='monthly', name=None))

In [ ]:
# for y, m, s in IterateOutputCESM(domain='atm', run='lpi', tavg='monthly'):
#     if m==1: print(y)

In [ ]:
%%time
yrly_avg_nc('ocn', 'ctrl', ['UVEL', 'VVEL'], test=False)
yrly_avg_nc('ocn', 'rcp' , ['UVEL', 'VVEL'], test=False)

In [ ]:
# %%time
# yrly_avg_nc('ocn', 'ctrl', ['TEMP', 'PD'], test=False)
# yrly_avg_nc('ocn', 'rcp' , ['TEMP', 'PD'], test=False)

In [ ]:
# %%time
# yrly_avg_nc('ocn', 'ctrl', ['SHF'], test=False)
# yrly_avg_nc('ocn', 'rcp', ['SHF'], test=False)

In [ ]:
# %%time
# yrly_avg_nc('atm', 'lpi', ['T', 'T850', 'U', 'V'], test=False)
# yrly_avg_nc('atm', 'rcp' , ['T', 'T850', 'U', 'V'], test=False)

In [ ]:
# # %%time
# yrly_avg_nc('atm', 'ctrl', ['FSNT', 'FLNT'], test=False)
# yrly_avg_nc('atm', 'rcp', ['FSNT', 'FLNT'], test=False)

# Statistical testing

In [ ]:
SST_yrly_detr_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc', decode_times=False)
AMO_ctrl = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_ctrl.nc', decode_times=False)

In [ ]:
X1 = SST_yrly_detr_ctrl[:, 1000,1000]
X2 = SST_yrly_detr_ctrl[:, 2000,1000]
X3 = SST_yrly_detr_ctrl[:, 1000,2000]

In [ ]:
X1.plot()
X2.plot()
X3.plot()

In [ ]:
def batch_act(X, m):
    """estimating autocorrelation time from batch method 
    X .. time series
    m .. number of batches
    """
    n = int(np.floor(len(X)/m))
    tau = 0
    for i in range(m):
        sm   = np.var(X[i*n:(i+1)*n]) 
        s    = np.var(X)
        tau += sm**2/s**2
#         print(tau)
    return tau

In [ ]:
for i in np.arange(1, 20):
    print(f'{i:2d}: {batch_act(X1.values, i):5.2f}, {batch_act(X2.values, i):5.2f}, {batch_act(X3.values, i):5.2f}')



In [ ]:
AMO_ctrl.plot()
chebychev(AMO_ctrl, 13).plot()


In [ ]:
def auto_correlate(X):
#     X_ac = np.correlate(X, X, mode='full')[len(X):]
#     print(np.var(X))
#     X_ac = (np.correlate(X, X, mode='full')/(np.var(X)))[len(X):]
    X_ac = np.zeros((len(X)-1))
    for t in np.arange(1,len(X)-1):
        X_shifted = np.zeros((len(X)))
        X_shifted[t:] = X[:-t]
        X_ac[t-1] = np.corrcoef(X, X_shifted)[0,1]
    tau_X = 1 + 2*np.sum(X_ac)
    return X_ac, tau_X

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
plt.plot(auto_correlate(X)[0])

## effective sample size

In [ ]:
n = 1000
for i in range(20):
    # np.random.seed(seed=42)
    X = np.random.normal(size=(n))
    X = X-X.mean()
    X10 = running_mean(X, 100)
    # X10 = X10-X10.mean()
#     plt.plot(X)
    # plt.plot(lowpass(X, 100)[49:-50])
#     plt.plot(np.arange(49,n-50), X10)

    X_ac, tau_X = auto_correlate(X)
    X10_ac, tau_X10 = auto_correlate(X10)
    print(i, n, int(n/tau_X), int(n/tau_X10))

In [ ]:
plt.hist(X)

In [ ]:
X_ac, tau_X = auto_correlate(X)
X10_ac, tau_X10 = auto_correlate(X10)

In [ ]:
print(X_ac.sum(),X10_ac.sum())

In [ ]:
plt.axhline(0)
plt.plot(X_ac[49:-50])
# plt.plot(X10_ac)

In [ ]:
print(n, n/tau_X, n/tau_X10)

In [ ]:
plt.scatter(X[49:-50], X10)
np.corrcoef(X[49:-50], X10)